# Python version of Black-Scholes Model

In [3]:
from scipy import stats
import math
import time

def BSM(S, K, T, V, r, div, cp):
    """
    S : S0 initial stock price
    K : Strike
    T : Exp time in years
    V : Volatility (sigma)
    r : Risk free rate
    div : Dividend or not
    cp : +1 / -1 for call / put
    """
    d1 = (math.log(S/K)+(r-div+0.5*math.pow(V,2))*T) / (V*math.sqrt(T))
    d2 = d1 - V*math.sqrt(T)
    
    optprice = cp * S * math.exp(-div*T)*stats.norm.cdf(cp*d1) - cp*K*math.exp(-r*T)*stats.norm.cdf(cp*d2)
    return optprice

In [4]:
%timeit BSM(50,50,0.4167,0.4,0.1,0,-1)

371 µs ± 64.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Cython version 

In [36]:
import pyximport
pyximport.install()
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [44]:
%%cython
cdef extern from "math.h" nogil:
    double exp(double)
    double log(double)
    double pow(double, double)
    double sqrt(double)
    double erf(double) # Error function

cdef double std_norm_cdf(double x):
    return 0.5 * (1 + erf(x / sqrt(2.0)))

cpdef BSM_C(double S, double K, double T, double V, double r, double div, double cp):
    cdef double d1, d2, optprice
    d1 = (log(S / K) + (r - div + 0.5 * pow(V, 2)) * T) / (V * sqrt(T))
    d2 = d1 - V * sqrt(T)
    optprice = cp * S * exp(-div * T) * std_norm_cdf(cp * d1) - cp * K * exp(-r * T) * std_norm_cdf(cp * d2)
    return optprice

In [45]:
%timeit BSM_C(50.0,50.0,0.4167,0.4,0.1,0.0,-1)

200 ns ± 6.43 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


#### Only 200 nanoseconds this time, huge improvement